# chainRec

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
sys.path.append('./src/')
from chainRec import chainRec
from dataset import Dataset
from tqdm import tqdm

DATA_DIR = "./data/"
MODEL_DIR = "./models/"
OUTPUT_DIR = "./results/"

## carga datos

In [2]:
test = Dataset('mercado_libre_test', n_stage=2)

Initializing dataset: mercado_libre_test
Successfully initialized!
1459964 interactions about 177070 users, 2102275 items, 2 stages are loaded!


In [3]:
train = Dataset('mercado_libre', n_stage=2)

Initializing dataset: mercado_libre
Successfully initialized!
3646946 interactions about 413163 users, 2102277 items, 2 stages are loaded!


## Settings modelo

In [4]:
myModel = chainRec(train.n_user, train.n_item, train.n_stage, train.DATA_NAME)
myModel.config_global(MODEL_CLASS="chainRec_uniform", HIDDEN_DIM=16, 
                              LAMBDA=0.1, LEARNING_RATE=0.001, BATCH_SIZE=256,
                              target_stage_id=(2-1))

## Entrenamiento

Se entrena el modelo ejecutando el código source de chainrec mediante la command line. Ejemplo del comando es

`python test_final_stage.py --dataset mercadolibre --method chainRec_uniform --nStage 2 --embedSize 16 --l2 0.1`

### Generación resultados

In [9]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, li, lj, s, logloss, optimizer, valiloss = myModel.model_constructor(myModel.n_user, myModel.n_item, myModel.n_stage, myModel.HIDDEN_DIM, myModel.LAMBDA, myModel.LEARNING_RATE)
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(session, MODEL_DIR + myModel.MODEL_NAME + ".model.ckpt")
    with open('out.csv', 'a') as f:
      for _k0 in tqdm(range(30000, test.n_user, 30)):
        user_list = list(range(_k0, (_k0 + 30)))
        np.savetxt(f, np.argpartition(np.array(s.eval(feed_dict={u: user_list}))[:,:,1], -10, axis=1)[:, -10:], delimiter=',', fmt="%d")


INFO:tensorflow:Restoring parameters from ./models/mercado_libre.chainRec_uniform.dim.16.lambda.0.1.model.ckpt
100%|██████████| 4903/4903 [4:47:10<00:00,  3.51s/it]


In [11]:
test.n_user

177070